<h2>Из чего же сделаны классы?</h2>

Теперь посмотрим на объект изнутри. Мы уже обратили внимание, что объект класса надо пересоздать после внесения изменения в класс. Получается, что каждый объект "носит" с собой все свои функции.<br>
Среди прочего, это связано с тем, что Питон работает со ссылками на переменные, а не самими переменными. Переменная в Питоне - это не классический "ящик" в котором хранится значение. Это ссылка на подобный ящик.<br>
Функция <i>dir</i> выдает список всех полей и методов объекта.

Исследуем объект нашего класса морфологии.

In [3]:
import pymorphy2 
import re 
import math
import numpy as np

In [4]:
# Другое имя класса, так как он обладает несколько иной функциональностью.
class FasterMorphology2:
    """ Класс для быстрого морфологического анализа текстов и их векторизации.
    """
    
    def __init__(self): # Функция инициализации объекта после его создания.
        self.morpho=pymorphy2.MorphAnalyzer()
        self.cash={}
        self.dictionary={} # Добавим словарь для запоминания, на каком месте вектора находится какая начальная форма.
        
    def analyzeWords(self, words):
        """ Проводит морфологический анализ списка токенов words.
            Возвращает список начальных форм слов.
        """
        res=[]
        for w in words:
            if w in self.cash: # Сперва ищем очередное слово в кеше.
                res.append(self.cash[w])
            else: # Если его там нет, проводим морфологический анализ и кешируем.
                r=self.morpho.parse(w)[0].normal_form
                res.append(r)
                self.cash[w]=r
                if r not in self.dictionary: # Также для каждой начальной формы запоминаем ее позицию в векторе.
                    self.dictionary[r]=len(self.dictionary)+1
        return res
    
    def analyzeText(self, text):
        """ Проводит морфологический анализ строки с текстом text. 
            Выделяет из нее слова, написанные русской кириллицей.
            Возвращает список начальных форм слов.
        """
        words=[w[0] for w in re.findall("([А-ЯЁа-яё]+(-[А-ЯЁа-яё]+)*)", textWP)]
        return self.analyzeWords(words)
        
    # Вообще-то тоже самое умеет Counter, но ему надо сперва привести слова к начальной форме.
    def vectorizeAsDict(self, words):
        """ Возвращает векторное разреженное представление текста в виде словаря.
            Текст передается как список токенов words.
            Вместо позиции для индексации используется само слово.
            Возвращает словарь с начальными формами в ключах и частотами этих форм.
        """
        vct={}
        for w in words: # Для каждого слова прповодим анализ.
            if w in self.cash:
                vct[self.cash[w]]=vct.get(self.cash[w], 0)+1 # Считаем частоты слов.
            else:
                r=self.morpho.parse(w)[0].normal_form
                res.append(r)
                self.cash[w]=r
                vct[r]=vct.get(r, 0)+1
                if r not in self.dictionary:
                    self.dictionary[r]=len(self.dictionary)
        return vct
    
    def clearDict(self):
        """ Очищает словарь. Вдруг надо пересчитать так как изменилась размерность пространства.
        """
        self.dictionary={}
    
    def formDict(self, texts):
        """ Сформировать словарь по тексту не формируя разметку текста.
        """
        for text in texts:
            for word in text:
                if word not in self.cash:
                    r=self.morpho.parse(w)[0].normal_form
                    self.cash[word]=r
                    if r not in self.dictionary:
                        self.dictionary[r]=len(self.dictionary)
    
    def vectorizeAsList(self, words):
        """ Возвращает векторное представление текста в виде плотного списка (включает нули).
            Текст передается как список токенов words.
            Позиция каждого слова в векторе определяется числом, хранимым в dictionary.
            Возвращает словарь с начальными формами в ключах и частотами этих форм.
        """
        # Сперва обновляем dictionary.
        for word in words:
            if word not in self.cash:
                r=self.morpho.parse(w)[0].normal_form
                self.cash[word]=r
                if r not in self.dictionary:
                    self.dictionary[r]=len(self.dictionary.keys())
        # Теперь, когда все слова есть в кеше и словаре и известен размер вектора, можно приступать к векторизации.
        vct=[0 for _ in self.dictionary]
        for word in words:
            vct[self.dictionary[self.cash[word]]]+=1
        return vct
    
    def vectorizeAsList2(self, words):
        """ Возвращает векторное представление текста в виде плотного списка (включает нули).
            Текст передается как список токенов words. В вектор включаются только слова, находящиес в словаре.
            Позиция каждого слова в векторе определяется числом, хранимым в dictionary.
            Возвращает словарь с начальными формами в ключах и частотами этих форм.
        """
        vct=[0 for _ in self.dictionary]
        for word in words:
            if word in self.cash:
                vct[self.dictionary[self.cash[word]]]+=1
        return vct

    def vectorizeAsArray(self, words):
        """ Возвращает векторное представление текста в виде плотного массива (включает нули).
            Текст передается как список токенов words.
            Позиция каждого слова в векторе определяется числом, хранимым в dictionary.
            Возвращает словарь с начальными формами в ключах и частотами этих форм.
        """
        # Сперва обновляем dictionary.
        for word in words:
            if word not in self.cash:
                r=self.morpho.parse(w)[0].normal_form
                self.cash[word]=r
                if r not in self.dictionary:
                    self.dictionary[r]=len(self.dictionary.keys())
        # Теперь, когда все слова есть в кеше и словаре и известен размер вектора, можно приступать к векторизации.
        vct=np.zeros((len(self.dictionary)))
        for word in words:
            vct[self.dictionary[self.cash[word]]]+=1
        return vct

    # Здесь мы заложили проблему. Функция не умеет считать расстояние между p.array.
    def cosineSimilarity(self, a, b):
        """ Функция расчета косинусной меры сходства между двумя векторными представлениями текста.
            Работает по-разному в зависимости от представления вектора.
        """
        if type(a)!=type(b): # Тип векторов должен совпадать.
            return None
        if isinstance(a, list): # Если это списки, значит это плотное представление вектора.
            if len(a)==0 or len(b)==0 or len(a)!=len(b): # Длины векторов в этом случае должны совпадать.
                return 0
            #sumab=sum([ai*bi for ai, bi in zip(a,b)])
            sumab=sum([a[na]*b[na] for na in range(len(a))])
            suma2=sum([a[na]*a[na] for na in range(len(a))])
            sumb2=sum([b[na]*b[na] for na in range(len(a))])
            return sumab/math.sqrt(suma2*sumb2)        
        elif isinstance(a, dict): # Разреженное представление вектора - хранятся только ненулевые значения.
            if len(a.keys())==0 or len(b.keys())==0: # Вектора должны хранить хоть что-то.
                return 0
            sumab=sum([a[na]*b[na] for na in set(a.keys())&set(b.keys())])
#            sumab=sum([a[na]*b[na] for na in a.keys() if na in b.keys()])
            suma2=sum([a[na]*a[na] for na in a.keys()])
            sumb2=sum([b[nb]*b[nb] for nb in b.keys()])
            return sumab/math.sqrt(suma2*sumb2)  
        return 0
    
    def JaccardCoefficient(self, a, b):
        """ Коэффициент Жаккара - отношение количества слов, встречающихся в обоих текстах к объединению лексики.
        """
        if type(a)!=type(b): # Тип векторов должен совпадать.
            return None
        if isinstance(a, list): # Если это списки, значит это плотное представление вектора.
            if len(a)==0 or len(b)==0 or len(a)!=len(b): # Длины векторов в этом случае должны совпадать.
                return 0
            union=len(a)-[aa*bb for aa, bb in zip(a, b)].count(0)
            intersection=len(a)-[aa+bb for aa, bb in zip(a, b)].count(0)
            return union/intersection
        elif isinstance(a, dict): # Разреженное представление вектора - хранятся только ненулевые значения.
            if len(a.keys())==0 or len(b.keys())==0: # Вектора должны хранить хоть что-то.
                return 0
            return len(set(a.keys())&set(b.keys()))/len(set(a.keys())|set(b.keys()))
        return 0
        

In [5]:
faster3 = FasterMorphology2()
dir(faster3)

['JaccardCoefficient',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'analyzeText',
 'analyzeWords',
 'cash',
 'clearDict',
 'cosineSimilarity',
 'dictionary',
 'formDict',
 'morpho',
 'vectorizeAsArray',
 'vectorizeAsDict',
 'vectorizeAsList',
 'vectorizeAsList2']

А поле \__dict\__ хранит только поля объекта. Но собственно хранит, а не содержит список названий.

In [6]:
faster3.__dict__

{'morpho': <pymorphy2.analyzer.MorphAnalyzer at 0x7f5e0f8ca080>,
 'cash': {},
 'dictionary': {}}

Поле `__class__` хранит ссылку на тип объекта.

In [7]:
type(faster3)

__main__.FasterMorphology2

In [8]:
faster3.__class__

__main__.FasterMorphology2

In [9]:
faster3.__class__.__name__

'FasterMorphology2'

То есть сам по себе тип является объектом и с ним можно точно так же работать.

In [10]:
type(faster3).__dict__

mappingproxy({'__module__': '__main__',
              '__doc__': ' Класс для быстрого морфологического анализа текстов и их векторизации.\n    ',
              '__init__': <function __main__.FasterMorphology2.__init__(self)>,
              'analyzeWords': <function __main__.FasterMorphology2.analyzeWords(self, words)>,
              'analyzeText': <function __main__.FasterMorphology2.analyzeText(self, text)>,
              'vectorizeAsDict': <function __main__.FasterMorphology2.vectorizeAsDict(self, words)>,
              'clearDict': <function __main__.FasterMorphology2.clearDict(self)>,
              'formDict': <function __main__.FasterMorphology2.formDict(self, texts)>,
              'vectorizeAsList': <function __main__.FasterMorphology2.vectorizeAsList(self, words)>,
              'vectorizeAsList2': <function __main__.FasterMorphology2.vectorizeAsList2(self, words)>,
              'vectorizeAsArray': <function __main__.FasterMorphology2.vectorizeAsArray(self, words)>,
          

Теперь добавим объекту несколько новых полей и функций и посмотрим как изменится список (при помощи оператора -).

In [11]:
faster5=FasterMorphology2()
faster4=FasterMorphology2()
faster4.dummy=0

In [12]:
set(dir(faster4))-set(dir(faster5))

{'dummy'}

Допустим, нам вдруг захотелось, чтобы faster4 начал считать Евклидово расстояние. Для этого добавим в объект соответствующую функцию.

In [13]:
def EuclideDistance(self, a, b):
    return math.sqrt(sum([aa*bb for aa, bb in zip(a, b)]))

faster4.EuclidianSimilarity=EuclideDistance

In [14]:
faster4.EuclidianSimilarity([1,2,3], [3,4,5])

TypeError: EuclideDistance() missing 1 required positional argument: 'b'

Что-то опять пошло не так. Оказывается в Питоне функции отличаются от методов класса.

In [15]:
print(faster4.analyzeWords)
print(faster4.EuclidianSimilarity)

<bound method FasterMorphology2.analyzeWords of <__main__.FasterMorphology2 object at 0x7f5e0f562860>>
<function EuclideDistance at 0x7f5e0e7eeb70>


Чтобы привязать метод к отдельному объекту необходимо вызвать специальную функцию.

In [16]:
import types

In [17]:
faster4.EuclidianSimilarity=types.MethodType(EuclideDistance, faster4)

In [18]:
def EuclideDistance(a, b):
    return math.sqrt(sum([aa*bb for aa, bb in zip(a, b)]))

faster4.EuclidianSimilarity=EuclideDistance

In [19]:
faster4.EuclidianSimilarity([1,2,3], [3,4,5])

5.0990195135927845

А вот поменять класс целиком довольно просто.

In [20]:
class forTests:

    def __init__(self):
        self.aha=0
        self.uhu=1

test1=forTests()
print("-- test1 -- ")
print(dir(test1))

def dummyFunc(self):
    return 0

forTests.dummy=dummyFunc

test2=forTests()
test2.dummy()

print("\n-- diff -- ")
print(set(dir(test1))-set(dir(test2)))
print("\n-- test1 -- ")
print(dir(test1))
print("\n-- test2 -- ")
print(dir(test2))


-- test1 -- 
['__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', 'aha', 'uhu']

-- diff -- 
set()

-- test1 -- 
['__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', 'aha', 'dummy', 'uhu']

-- test2 -- 
['__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__

Но обратите внимание, метод появился теперь у всех объектов данного класса.

In [27]:
type(type(0)).__class__.__name__


'type'

In [28]:
dir(type(0))

['__abs__',
 '__add__',
 '__and__',
 '__bool__',
 '__ceil__',
 '__class__',
 '__delattr__',
 '__dir__',
 '__divmod__',
 '__doc__',
 '__eq__',
 '__float__',
 '__floor__',
 '__floordiv__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getnewargs__',
 '__gt__',
 '__hash__',
 '__index__',
 '__init__',
 '__init_subclass__',
 '__int__',
 '__invert__',
 '__le__',
 '__lshift__',
 '__lt__',
 '__mod__',
 '__mul__',
 '__ne__',
 '__neg__',
 '__new__',
 '__or__',
 '__pos__',
 '__pow__',
 '__radd__',
 '__rand__',
 '__rdivmod__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__rfloordiv__',
 '__rlshift__',
 '__rmod__',
 '__rmul__',
 '__ror__',
 '__round__',
 '__rpow__',
 '__rrshift__',
 '__rshift__',
 '__rsub__',
 '__rtruediv__',
 '__rxor__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__sub__',
 '__subclasshook__',
 '__truediv__',
 '__trunc__',
 '__xor__',
 'bit_length',
 'conjugate',
 'denominator',
 'from_bytes',
 'imag',
 'numerator',
 'real',
 'to_bytes']

<h2>Перегрузка операторов</h2>

Давайте продолжим развивать наш учебный класс.<br>
Еще раз внимательно посмотрим на список методов нашего класса.

In [22]:
dir(FasterMorphology2)

['JaccardCoefficient',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'analyzeText',
 'analyzeWords',
 'clearDict',
 'cosineSimilarity',
 'formDict',
 'vectorizeAsArray',
 'vectorizeAsDict',
 'vectorizeAsList',
 'vectorizeAsList2']

А что это за функции с двумя подчеркиваниями? И все ли из них такие private? <br>
На самом деле нет. Часть из этих функций - это синонимы для операторов. Мы ведь можем складывать два множества или матрицы. Каждый раз когда мы пишем<br>
a=b+c<br>
на самом деле вызывается следующий код.<br>
a=\__add\__(b,c)<br>
Операторы являются удобным представлением вызовов функций. Для того, чтобы определить соответствующий оператор надо просто добавить функцию в соответствующий класс. Список всех возможных операторов записан <a href="https://docs.python.org/3.7/library/operator.html">здесь</a> и <a href="https://docs.python.org/3/reference/datamodel.html#special-method-names">здесь</a>.<br>
То есть каждый класс может завести себе, например, оператор сложения, если ему это необходимо.<br>
Давайте немного преобразим наш класс. Пусть один объект можно будет сложить с другим, после чего у него пополнится словарь. А в наш класс можно будет отправить строку при помощи оператора <<, а оператор вернет векторное представление. И еще кое-что разной степени приятности.

In [23]:
# Вся перегрузка операторов находится внизу класса.
class FasterMorphology2:
    """ Класс для быстрого морфологического анализа текстов и их векторизации.
    """
    
    def __init__(self): # Функция инициализации объекта после его создания.
        self.morpho=pymorphy2.MorphAnalyzer()
        self.cash={}
        self.dictionary={} # Добавим словарь для запоминания, на каком месте вектора находится какая начальная форма.
        
    def analyzeWords(self, words):
        """ Проводит морфологический анализ списка токенов words.
            Возвращает список начальных форм слов.
        """
        res=[]
        for w in words:
            if w in self.cash: # Сперва ищем очередное слово в кеше.
                res.append(self.cash[w])
            else: # Если его там нет, проводим морфологический анализ и кешируем.
                r=self.morpho.parse(w)[0].normal_form
                res.append(r)
                self.cash[w]=r
                if r not in self.dictionary: # Также для каждой начальной формы запоминаем ее позицию в векторе.
                    self.dictionary[r]=len(self.dictionary)
        return res
    
    def breakByWords(self, text):
        """ Разбивает текст на русские слова.
        """
        return [w[0].lower() for w in re.findall("([А-ЯЁа-яё]+(-[А-ЯЁа-яё]+)*)", text)]
    
    def analyzeText(self, text):
        """ Проводит морфологический анализ строки с текстом text. 
            Выделяет из нее слова, написанные русской кириллицей.
            Возвращает список начальных форм слов.
        """
        words=self.breakByWords(text)
        return self.analyzeWords(words)
        
    # Вообще-то тоже самое умеет Counter, но ему надо сперва привести слова к начальной форме.
    def vectorizeAsDict(self, words):
        """ Возвращает векторное разреженное представление текста в виде словаря.
            Текст передается как список токенов words.
            Вместо позиции для индексации используется само слово.
            Возвращает словарь с начальными формами в ключах и частотами этих форм.
        """
        # Если это был текст - разбиваем на слова.
        if isinstance(words, str):
            words=self.breakByWords(words)

        vct={}
        for word in words: # Для каждого слова прповодим анализ.
            if word in self.cash:
                vct[self.cash[word]]=vct.get(self.cash[word], 0)+1 # Считаем частоты слов.
            else:
                r=self.morpho.parse(word)[0].normal_form
                res.append(r)
                self.cash[word]=r
                vct[r]=vct.get(r, 0)+1
                if r not in self.dictionary:
                    self.dictionary[r]=len(self.dictionary)
        return vct
    
    def clearDict(self):
        """ Очищает словарь. Вдруг надо пересчитать так как изменилась размерность пространства.
        """
        self.dictionary={}
    
    def formDict(self, texts):
        """ Сформировать словарь по тексту не формируя разметку текста.
        """
        for text in texts:
            for word in text:
                if word not in self.cash:
                    r=self.morpho.parse(word)[0].normal_form
                    self.cash[word]=r
                    if r not in self.dictionary:
                        self.dictionary[r]=len(self.dictionary)
    
    def vectorizeAsList(self, words2):
        """ Возвращает векторное представление текста в виде плотного списка (включает нули).
            Текст передается как список токенов words.
            Позиция каждого слова в векторе определяется числом, хранимым в dictionary.
            Возвращает словарь с начальными формами в ключах и частотами этих форм.
        """
        # Если это был текст - разбиваем на слова.
        if isinstance(words2, str):
            words=self.breakByWords(words2)
        else:
            words=words2

        # Сперва обновляем dictionary.
        for word in words:
            if word not in self.cash:
                r=self.morpho.parse(word)[0].normal_form
                self.cash[word]=r
                if r not in self.dictionary:
                    self.dictionary[r]=len(self.dictionary.keys())
        # Теперь, когда все слова есть в кеше и словаре и известен размер вектора, можно приступать к векторизации.
        vct=[0 for _ in self.dictionary]
        for word in words:
            vct[self.dictionary[self.cash[word]]]+=1
        return vct
    
    def vectorizeAsList2(self, words):
        """ Возвращает векторное представление текста в виде плотного списка (включает нули).
            Текст передается как список токенов words. В вектор включаются только слова, находящиес в словаре.
            Позиция каждого слова в векторе определяется числом, хранимым в dictionary.
            Возвращает словарь с начальными формами в ключах и частотами этих форм.
        """
        vct=[0 for _ in self.dictionary]
        for word in words:
            if word in self.cash:
                vct[self.dictionary[self.cash[word]]]+=1
        return vct

    def vectorizeAsArray(self, words):
        """ Возвращает векторное представление текста в виде плотного массива (включает нули).
            Текст передается как список токенов words.
            Позиция каждого слова в векторе определяется числом, хранимым в dictionary.
            Возвращает словарь с начальными формами в ключах и частотами этих форм.
        """
        # Если это был текст - разбиваем на слова.
        if isinstance(words, str):
            words=self.breakByWords(words)

        # Сперва обновляем dictionary.
        for word in words:
            if word not in self.cash:
                r=self.morpho.parse(w)[0].normal_form
                self.cash[word]=r
                if r not in self.dictionary:
                    self.dictionary[r]=len(self.dictionary.keys())
        # Теперь, когда все слова есть в кеше и словаре и известен размер вектора, можно приступать к векторизации.
        vct=np.zeros((len(self.dictionary)))
        for word in words:
            vct[self.dictionary[self.cash[word]]]+=1
        return vct

    # Здесь мы заложили проблему. Функция не умеет считать расстояние между p.array.
    def cosineSimilarity(self, a, b):
        """ Функция расчета косинусной меры сходства между двумя векторными представлениями текста.
            Работает по-разному в зависимости от представления вектора.
        """
        if type(a)!=type(b): # Тип векторов должен совпадать.
            return None
        if isinstance(a, list): # Если это списки, значит это плотное представление вектора.
            if len(a)==0 or len(b)==0 or len(a)!=len(b): # Длины векторов в этом случае должны совпадать.
                return 0
            sumab=sum([a[na]*b[na] for na in range(len(a))])
            suma2=sum([a[na]*a[na] for na in range(len(a))])
            sumb2=sum([b[na]*b[na] for na in range(len(a))])
            return sumab/math.sqrt(suma2*sumb2)        
        elif isinstance(a, dict): # Разреженное представление вектора - хранятся только ненулевые значения.
            if len(a.keys())==0 or len(b.keys())==0: # Вектора должны хранить хоть что-то.
                return 0
            sumab=sum([a[na]*b[na] for na in set(a.keys())&set(b.keys())])
#            sumab=sum([a[na]*b[na] for na in a.keys() if na in b.keys()])
            suma2=sum([a[na]*a[na] for na in a.keys()])
            sumb2=sum([b[nb]*b[nb] for nb in b.keys()])
            return sumab/math.sqrt(suma2*sumb2)  
        return 0
    
    def JaccardCoefficient(self, a, b):
        """ Коэффициент Жаккара - отношение количества слов, встречающихся в обоих текстах к объединению лексики.
        """
        if type(a)!=type(b): # Тип векторов должен совпадать.
            return None
        if isinstance(a, list): # Если это списки, значит это плотное представление вектора.
            if len(a)==0 or len(b)==0 or len(a)!=len(b): # Длины векторов в этом случае должны совпадать.
                return 0
            union=len(a)-[aa*bb for aa, bb in zip(a, b)].count(0)
            intersection=len(a)-[aa+bb for aa, bb in zip(a, b)].count(0)
            return union/intersection
        elif isinstance(a, dict): # Разреженное представление вектора - хранятся только ненулевые значения.
            if len(a.keys())==0 or len(b.keys())==0: # Вектора должны хранить хоть что-то.
                return 0
            return len(set(a.keys())&set(b.keys()))/len(set(a.keys())|set(b.keys()))
        return 0
        
    def __iadd__(self, other):
        """ Оператор добавления словаря от другого объекта.
        """
        self.cash.update(other.cash)
        for word in set(other.dictionary.keys())-set(self.dictionary.keys()):
            self.dictionary[word]=len(self.dictionary)
        return self
    
    def __lshift__(self, text):
        """ Оператор возвращает векторное представление текста
        """
        return self.vectorizeAsList(text)
        
    def __repr__(self):
        """ Текстовое представление объекта.
        """
        return "Object of class FasterMorphology2 <"+str(id(self))+ \
                ">\nCash size: "+str(len(self.cash))+"\nDictionary size: "+str(len(self.dictionary))
    
    def __getitem__(self, key):
        """ Возвращает элемент словаря при помощи квадратных скобок.
        """
        if isinstance(key, slice):
            return list(self.dictionary.keys())[key.start: key.stop: key.step]
        else:
            return list(self.dictionary.keys())[key]
    
    def __bool__(self):
        """ Класс ведет себя как булевская переменная. Проверяет было ли что-нибудь закешировано.
        """
        return len(self.dictionary)!=0
    
    def __call__(self):
        """ Объект класса можно "вызвать" как функцию. Можно просто переопределить оператор "круглые скобки".
        """
        print("-=* Overall results for FasterMorphology2*=-\nCash size: "+str(len(self.cash))+"\nDictionary size: "+str(len(self.dictionary)))
        

Итак, мы добавили некоторые операторы в наш класс. Теперь опробуем их на трех произведениях.

In [24]:
5 is 5

True

In [25]:
with open("data/war_and_peace.txt", encoding="utf8") as fil:
    textWP=fil.read()

with open("data/Kard_Orson__Igra_Jendera.fb2") as fil:
    textEnd=fil.read()
    
textMart=""
for i in range(2, 33):
    with open("data/veyr/index_split_0"+"{:0>2}".format(i)+".xhtml") as fil:
        textMart+=fil.read()


Сперва сделаем по старой технологии, с вызовом функций.

In [26]:
fasterWP=FasterMorphology2()
fasterMart=FasterMorphology2()

In [27]:
%time vctWP=fasterWP.vectorizeAsList(textWP)
%time vctMart=fasterMart.vectorizeAsList(textMart)

CPU times: user 1.55 s, sys: 31.9 ms, total: 1.58 s
Wall time: 1.62 s
CPU times: user 387 ms, sys: 0 ns, total: 387 ms
Wall time: 387 ms


In [28]:
%%time
fasterEnd=FasterMorphology2()
vctEnd=fasterEnd.analyzeText(textEnd)

CPU times: user 381 ms, sys: 14.9 ms, total: 396 ms
Wall time: 393 ms


А теперь попробуем выполнить тоже самое, но при помощи новых перегруженных операторов. 

In [29]:
%%time
fasterEnd=FasterMorphology2()
print(fasterEnd) # Выводим объект.
if not fasterEnd: # Проверяем есть ли что-то в словаре.
    fasterEnd+=fasterWP # Пополняем словарь.
    fasterEnd+=fasterMart
print(fasterEnd)
vctEnd=fasterEnd<<textEnd # Векторизуем текст.
fasterEnd() # Вызываем метод от объекта.

Object of class FasterMorphology2 <139813220091384>
Cash size: 0
Dictionary size: 0
Object of class FasterMorphology2 <139813220091384>
Cash size: 59297
Dictionary size: 21805
-=* Overall results for FasterMorphology2*=-
Cash size: 65696
Dictionary size: 23707
CPU times: user 387 ms, sys: 15.3 ms, total: 403 ms
Wall time: 400 ms


Обратите внимание, что не все функции являются собственно операторами. Некоторые из них - это специальные функции, вызываемые в специальных обстоятельствах.

<h2>Значения параметров по умолчанию</h2>

Теперь попробуем разобраться с передачей параметров в функции.<br>
Помните функцию, которая выбирала "значимые части речи" с ее довольно спорным списком частей речи? Было бы здорово, если бы пользователь мог передавать туда свой список частей речи, но при это неспециалист мог бы просто согласиться со списком автора функции.<br>
Для этого существуют значения параметров по умолчанию, которые прописываются в объявлении функции.

In [30]:
# Обратите внимание на присвоение значения posList - именно так задается значение по умолчанию.
def getMeaningfullWords(morph, text, posList=('ADJF', 'NOUN', 'VERB')):
    words=[]
    tokens=re.findall('[А-Яа-яЁё]+\-[А-Яа-яЁё]+|[А-Яа-яЁё]+', text)
    for t in tokens:
        pv=morph.parse(t)
        if pv[0].tag.POS in posList:
            words.append(pv[0].normal_form)
    return words

morph=pymorphy2.MorphAnalyzer()

Теперь мы можем просто вызвать функцию со своим списком частей речи.

In [31]:
getMeaningfullWords(morph, textMart[1000:1500], ['ADJF', 'NOUN', 'VERB', 'PREP'])

['месяц',
 'самый',
 'значительный',
 'в',
 'жизнь',
 'обернуться',
 'кошмар',
 'знать',
 'прочесть',
 'этот',
 'строка',
 'думать',
 'в',
 'конец',
 'конец',
 'мой',
 'запись',
 'весь',
 'найти',
 'мочь',
 'год',
 'через',
 'сто',
 'для',
 'отчёт',
 'на',
 'шесть',
 'сутки',
 'погибнуть',
 'наш',
 'команда',
 'счесть',
 'мёртвый',
 'мочь',
 'мочь',
 'в',
 'мой',
 'честь',
 'объявить',
 'день',
 'национальный',
 'траур',
 'на',
 'мой',
 'страница',
 'в',
 'википедия',
 'появиться',
 'запись',
 'марк',
 'уотня']

А можем согласиться на список авторов функции и не передавать ничего.

In [32]:
getMeaningfullWords(morph, textMart[1000:1500])

['месяц',
 'самый',
 'значительный',
 'жизнь',
 'обернуться',
 'кошмар',
 'знать',
 'прочесть',
 'этот',
 'строка',
 'думать',
 'конец',
 'конец',
 'мой',
 'запись',
 'весь',
 'найти',
 'мочь',
 'год',
 'сто',
 'отчёт',
 'шесть',
 'сутки',
 'погибнуть',
 'наш',
 'команда',
 'счесть',
 'мёртвый',
 'мочь',
 'мочь',
 'мой',
 'честь',
 'объявить',
 'день',
 'национальный',
 'траур',
 'мой',
 'страница',
 'википедия',
 'появиться',
 'запись',
 'марк',
 'уотня']

Передача параметров может быть чрезвычайно удобна в нескольких случаях.
- В подавляющем большинстве случаев функция вызывается с значениями параметров по умолчанию. 
- Вам необходимо переделать дизайн функции так, чтобы в ней появились новые параметры, но при этом необходимо чтобы весь остальной код продолжал выполняться.
- Необходимо застраховаться от ошибки.
- Параметр по умолчанию принимает особое значение, которое нам о чем-то говорит.

Можно сделать несколько параметров со значениями по умолчанию. Если при этом не передавать последовательно несколько значений с конца, то можно просто их не писать. Если мы пропустили значение из середины списка, для остальных придется писать какому параметру какое значение передается.

In [33]:
def dummie1(a=0, b=1, c=2, d=3):
    return a+b*c-d

In [34]:
print(dummie1())
print(dummie1(2))
print(dummie1(2, 3))
print(dummie1(2, d=4))
print(dummie1.__defaults__)
print(dummie1.__code__)

-1
1
5
0
(0, 1, 2, 3)
<code object dummie1 at 0x7f28cbc0ef60, file "<ipython-input-33-103ef1c2c247>", line 1>


В книге Л. Рамальо "Python: к вершинам мастерства" есть раздел с названием "Значения по умолчанию изменяемого типа: плохая идея" (с. 259). В ней приведен следующий пример. Пусть у нас есть класс автобуса, который хранит имена пассажиров.

In [32]:
class HauntedBus:
    """A bus model haunted by ghost passengers"""

    def __init__(self, passengers=[]):  # Вот здесь сделана ошибка, от которой потом все беды.
        self.passengers = passengers  

#     def __init__(self, passengers=None):  # Версия без ошибок.
#         if passengers:
#             self.passengers = passengers  
#         else:
#             self.passengers = []

    def pick(self, name):
        self.passengers.append(name)  

    def drop(self, name):
        self.passengers.remove(name)

        
bus1 = HauntedBus(['Alice', 'Bill'])
print(bus1.passengers)
#['Alice', 'Bill']
bus1.pick('Charlie')
bus1.drop('Alice')
print(bus1.passengers)
#['Bill', 'Charlie']
bus2 = HauntedBus()
bus2.pick('Carrie')
print(bus2.passengers)
#['Carrie']
bus3 = HauntedBus()
print(bus3.passengers)
#['Carrie']
bus3.pick('Dave')
print(bus2.passengers)
#['Carrie', 'Dave']
print(bus2.passengers is bus3.passengers)
#True
print(bus1.passengers)
#['Bill', 'Charlie']
print(dir(HauntedBus.__init__))
#['__annotations__', '__call__', ..., '__defaults__', ...]
print(HauntedBus.__init__.__defaults__) # Список значений по умолчанию для конструктора класса.
#(['Carrie', 'Dave'],)
print(HauntedBus.__init__.__defaults__[0] is bus2.passengers) # Оказывается всем спискам присвоена ссылка на объект по умолчанию!
#True
bus4 = HauntedBus()
print(bus4.passengers)
# ['Carrie', 'Dave']

['Alice', 'Bill']
['Bill', 'Charlie']
['Carrie']
['Carrie']
['Carrie', 'Dave']
True
['Bill', 'Charlie']
['__annotations__', '__call__', '__class__', '__closure__', '__code__', '__defaults__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__get__', '__getattribute__', '__globals__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__kwdefaults__', '__le__', '__lt__', '__module__', '__name__', '__ne__', '__new__', '__qualname__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__']
(['Carrie', 'Dave'],)
True
['Carrie', 'Dave']


Разбирая этот пример мы можем увидеть, что если в конструкторе список пассажиров берется как значение по умолчанию, то списку пассажиров данного объекта присваивается ссылка на список по умолчанию (который сам по себе тоже переменная, точнее, ее значение). Соответственно, в дальнейшем все автобусы, которым мы не передали список, будут указывать на один и тот же список. <br>
Будьте бдительны! Делайте глубокую копию со значений по умолчанию изменяемого типа.

<h2>Произвольный список параметров функции</h2>

Теперь давайте разеремся как передать в функцию произвольное количество параметров.<br>
В Python можно передать переменное количество аргументов двумя способами:
- кортеж `*args` для неименованных аргументов;
- словарь `**kwargs` для именованных аргументов.

Мы используем `*args` и `**kwargs` в качестве аргумента, когда заранее не известно, сколько значений мы хотим передать функции. Функция может принимать оба этих параметра на случай, если передача будет вестись как с именем, так и без.

In [36]:
def mult(*multipliers):
    print(multipliers)
    res=1
    for arg in multipliers:
        res*=arg
    return res

def HTMLizer(tag, *text, **args):
    ar=', '.join([a[0]+'="'+str(a[1])+'"' for a in args.items()])
    txt=' '.join([str(t) for t in text])
    return "<"+tag+" "+ar+">"+txt+"</"+tag+">"

In [37]:
print(mult(2, 3, 4, 5))
print(HTMLizer("p", "that is", "some text", size=12, color="red"))
print(HTMLizer("p"))

(2, 3, 4, 5)
120
<p size="12", color="red">that is some text</p>
<p ></p>


Обратите внимание, что при передачче параметра знак звездочки означает распаковку параметров. Например, функция принимает несколько параметров. Вместо этого Вы передаете кортеж с тем же числом полей, добавив к нему знак звездочки. 

In [38]:
asd=(2, 3, 4, 5)
mult(*asd)

(2, 3, 4, 5)


120

In [36]:
def dummie2(a, b):
    print(10*a+b)

ddd=('1', '2')
dummie2(*ddd)

fff={'b':2, 'a':5}
dummie2(**fff)# dummie2(a=5, b=2)
print(tuple(fff))

11111111112
52
('b', 'a')


In [37]:
ddd = {'a':1, 's':3}
for d in ddd:
    print(d)

a
s


Это работает не только как в примере с произвольными параметрами, но и на любых других функциях.

In [40]:
def printCoords(long, lat):
    print("Longitude: ", long, ", latitude: ", lat)
    
coords=[(1.23, 2.34), (3.45, 6.45), (6.76, 8.98)]
for coord in coords:
    printCoords(*coord)

Longitude:  1.23 , latitude:  2.34
Longitude:  3.45 , latitude:  6.45
Longitude:  6.76 , latitude:  8.98


Также можно принять в функцию список аргументов по имени и в зависимости от того, что в нем есть производить те или иные действия.

In [41]:
def printCoords(long, lat, **kwargs):
    res=""
    if 'planet' in kwargs.keys():
        res+='Coordinates on planet '+kwargs['planet']+" are "
    res+="Longitude: "+str(long)+", Latitude: "+str(lat)
    print(res)
    
coords=[(1.23, 2.34), (3.45, 6.45), (6.76, 8.98)]
planets=["Earth", "Mars", "Krypton"]
for coord in coords:
    printCoords(*coord)
for coord, plnt in zip(coords, planets):
    printCoords(*coord, planet=plnt, strange='high')

Longitude: 1.23, Latitude: 2.34
Longitude: 3.45, Latitude: 6.45
Longitude: 6.76, Latitude: 8.98
Coordinates on planet Earth are Longitude: 1.23, Latitude: 2.34
Coordinates on planet Mars are Longitude: 3.45, Latitude: 6.45
Coordinates on planet Krypton are Longitude: 6.76, Latitude: 8.98


<h2>Декораторы функций</h2>

Теперь перейдем к декораторам.<br>
Иногда нам необходимо расширить возможности какой-то функции. Для этих целей в Питоне используются декораторы, то есть функции, которые принимают другую функцию и возвращают третью (в общем случае).

In [42]:
def trace(func):
    def inner(*args, **kwargs):
        print("calling function:", func.__name__, ", parameters", args, kwargs)
        return func(*args, **kwargs)
    return inner

@trace
def calc(a, b):
    return a+b

calc(1,2)

calling function: calc , parameters (1, 2) {}


3

Обратите внимание, что функций в общем случае три:
- декорируемая;
- декорирующая;
- та, которой декарируют.

Общая идея декоратора состоит в том, что мы можем некоторым образом попросить вторую функцию сделать так, чтобы вместо первой вызывалась третья. При этом третья знает о существовании первой и может использует результаты ее работы.<br>
В некотором роде, мы не заявляем, что декорируем первой функцией. Мы скромно просим задекорировать для нас первую функцию.

Задекорировать функцию можно несколькими другими функциями. Порядок декораторов имеет значение.

In [43]:
# https://compscicenter.ru/media/slides/python_2015_autumn/2015_09_21_python_2015_autumn_93c2yAw.pdf

def square(func):
    return lambda x: func(x * x)

def addsome(func):
    return lambda x: func(x + 42)

@square
@addsome
def identity(x):
    return x

print(identity(2))
# 46
@addsome
@square
def identity(x):
    return x
print(identity(2))
# 1936

46
1936


Важное замечание про декораторы. <br>
Декорирующие функции выполняются по мере объявления.

In [44]:
# https://compscicenter.ru/media/slides/python_2015_autumn/2015_09_21_python_2015_autumn_93c2yAw.pdf

def square(func):
    print("square")
    return lambda x: func(x * x)

def addsome(func):
    print("addsome")
    return lambda x: func(x + 42)

@square
@addsome
def identity1(x):
    print("identity 1")
    return x

print("-----")

@addsome
@square
def identity2(x):
    print("identity 2")
    return x

addsome
square
-----
square
addsome


In [45]:
print(identity1(2))
# 46

print(identity2(2))
# 1936

identity 1
46
identity 2
1936


Получается, что место, где декорируется функция, например, вот такое.

`
@first
def second():
    pass
`

Так вот в реальности оно выглядит вот так.

`
def second():
    pass
second=first(second)
`


Теперь посмотрим на библиотеку с декораторами `functools`. Например, `functools.lru_cache` кеширует результаты и при повторном вызове с теми же параметрами подставляет результаты из кеша.

In [46]:
import functools

In [47]:
@trace
def fibonacci(n): # Посчитаем числа Фибоначи.
    if n<2:
        return n
    return fibonacci(n-2)+fibonacci(n-1)

@trace
@functools.lru_cache()
def fibonacci2(n):
    if n<2:
        return n
    return fibonacci2(n-2)+fibonacci2(n-1)

@functools.lru_cache()
@trace
def fibonacci3(n):
    if n<2:
        return n
    return fibonacci3(n-2)+fibonacci3(n-1)


In [48]:
print(fibonacci(6))
print("-----")
print(fibonacci2(6))
print("-----")
print(fibonacci3(6))

calling function: fibonacci , parameters (6,) {}
calling function: fibonacci , parameters (4,) {}
calling function: fibonacci , parameters (2,) {}
calling function: fibonacci , parameters (0,) {}
calling function: fibonacci , parameters (1,) {}
calling function: fibonacci , parameters (3,) {}
calling function: fibonacci , parameters (1,) {}
calling function: fibonacci , parameters (2,) {}
calling function: fibonacci , parameters (0,) {}
calling function: fibonacci , parameters (1,) {}
calling function: fibonacci , parameters (5,) {}
calling function: fibonacci , parameters (3,) {}
calling function: fibonacci , parameters (1,) {}
calling function: fibonacci , parameters (2,) {}
calling function: fibonacci , parameters (0,) {}
calling function: fibonacci , parameters (1,) {}
calling function: fibonacci , parameters (4,) {}
calling function: fibonacci , parameters (2,) {}
calling function: fibonacci , parameters (0,) {}
calling function: fibonacci , parameters (1,) {}
calling function: fi

Теперь попробуем аккуратно заделать недостаток нашего класса, который состоит в том, что мы не умеем считать косинусную меру для `numpy.array`. Будем использовать для этого `functools.singledispatch`.

In [49]:
# Все декораторы вынесены из определения класса, так как не дело это класса считать косинусную меру. Она сама по себе.
# А еще я заменил действие оператора << , теперь он возвращает словарь. Так все-таки проще считать косинусную меру.
class FasterMorphology2:
    """ Класс для быстрого морфологического анализа текстов и их векторизации.
    """
    
    def __init__(self): # Функция инициализации объекта после его создания.
        self.morpho=pymorphy2.MorphAnalyzer()
        self.cash={}
        self.dictionary={} # Добавим словарь для запоминания, на каком месте вектора находится какая начальная форма.
        
    def analyzeWords(self, words):
        """ Проводит морфологический анализ списка токенов words.
            Возвращает список начальных форм слов.
        """
        res=[]
        for w in words:
            if w in self.cash: # Сперва ищем очередное слово в кеше.
                res.append(self.cash[w])
            else: # Если его там нет, проводим морфологический анализ и кешируем.
                r=self.morpho.parse(w)[0].normal_form
                res.append(r)
                self.cash[w]=r
                if r not in self.dictionary: # Также для каждой начальной формы запоминаем ее позицию в векторе.
                    self.dictionary[r]=len(self.dictionary)
        return res
    
    def breakByWords(self, text):
        """ Разбивает текст на русские слова.
        """
        return [w[0].lower() for w in re.findall("([А-ЯЁа-яё]+(-[А-ЯЁа-яё]+)*)", text)]
    
    def analyzeText(self, text):
        """ Проводит морфологический анализ строки с текстом text. 
            Выделяет из нее слова, написанные русской кириллицей.
            Возвращает список начальных форм слов.
        """
        words=self.breakByWords(text)
        return self.analyzeWords(words)
        
    # Вообще-то тоже самое умеет Counter, но ему надо сперва привести слова к начальной форме.
    def vectorizeAsDict(self, words):
        """ Возвращает векторное разреженное представление текста в виде словаря.
            Текст передается как список токенов words.
            Вместо позиции для индексации используется само слово.
            Возвращает словарь с начальными формами в ключах и частотами этих форм.
        """
        # Если это был текст - разбиваем на слова.
        if isinstance(words, str):
            words=self.breakByWords(words)

        vct={}
        for word in words: # Для каждого слова прповодим анализ.
            if word in self.cash:
                vct[self.cash[word]]=vct.get(self.cash[word], 0)+1 # Считаем частоты слов.
            else:
                r=self.morpho.parse(word)[0].normal_form
                self.cash[word]=r
                vct[r]=vct.get(r, 0)+1
                if r not in self.dictionary:
                    self.dictionary[r]=len(self.dictionary)
        return vct
    
    def clearDict(self):
        """ Очищает словарь. Вдруг надо пересчитать так как изменилась размерность пространства.
        """
        self.dictionary={}
    
    def formDict(self, texts):
        """ Сформировать словарь по тексту не формируя разметку текста.
        """
        for text in texts:
            for word in text:
                if word not in self.cash:
                    r=self.morpho.parse(word)[0].normal_form
                    self.cash[word]=r
                    if r not in self.dictionary:
                        self.dictionary[r]=len(self.dictionary)
    
    def vectorizeAsList(self, words2):
        """ Возвращает векторное представление текста в виде плотного списка (включает нули).
            Текст передается как список токенов words.
            Позиция каждого слова в векторе определяется числом, хранимым в dictionary.
            Возвращает словарь с начальными формами в ключах и частотами этих форм.
        """
        # Если это был текст - разбиваем на слова.
        if isinstance(words2, str):
            words=self.breakByWords(words2)
        else:
            words=words2

        # Сперва обновляем dictionary.
        for word in words:
            if word not in self.cash:
                r=self.morpho.parse(word)[0].normal_form
                self.cash[word]=r
                if r not in self.dictionary:
                    self.dictionary[r]=len(self.dictionary.keys())
        # Теперь, когда все слова есть в кеше и словаре и известен размер вектора, можно приступать к векторизации.
        vct=[0 for _ in self.dictionary]
        for word in words:
            vct[self.dictionary[self.cash[word]]]+=1
        return vct
    
    def vectorizeAsList2(self, words):
        """ Возвращает векторное представление текста в виде плотного списка (включает нули).
            Текст передается как список токенов words. В вектор включаются только слова, находящиес в словаре.
            Позиция каждого слова в векторе определяется числом, хранимым в dictionary.
            Возвращает словарь с начальными формами в ключах и частотами этих форм.
        """
        vct=[0 for _ in self.dictionary]
        for word in words:
            if word in self.cash:
                vct[self.dictionary[self.cash[word]]]+=1
        return vct

    def vectorizeAsArray(self, words):
        """ Возвращает векторное представление текста в виде плотного массива (включает нули).
            Текст передается как список токенов words.
            Позиция каждого слова в векторе определяется числом, хранимым в dictionary.
            Возвращает словарь с начальными формами в ключах и частотами этих форм.
        """
        # Если это был текст - разбиваем на слова.
        if isinstance(words, str):
            words=self.breakByWords(words)

        # Сперва обновляем dictionary.
        for word in words:
            if word not in self.cash:
                r=self.morpho.parse(w)[0].normal_form
                self.cash[word]=r
                if r not in self.dictionary:
                    self.dictionary[r]=len(self.dictionary.keys())
        # Теперь, когда все слова есть в кеше и словаре и известен размер вектора, можно приступать к векторизации.
        vct=np.zeros((len(self.dictionary)))
        for word in words:
            vct[self.dictionary[self.cash[word]]]+=1
        return vct

    def __iadd__(self, other):
        """ Оператор добавления словаря от другого объекта.
        """
        self.cash.update(other.cash)
        for word in set(other.dictionary.keys())-set(self.dictionary.keys()):
            self.dictionary[word]=len(self.dictionary)
        return self
    
    def __lshift__(self, text):
        """ Оператор возвращает векторное представление текста в виде словаря.
        """
        return self.vectorizeAsDict(text)
        
    def __repr__(self):
        """ Текстовое представление объекта.
        """
        return "Object of class FasterMorphology2 <"+str(id(self))+ \
                ">\nCash size: "+str(len(self.cash))+"\nDictionary size: "+str(len(self.dictionary))
    
    def __getitem__(self, key):
        """ Возвращает элемент словаря при помощи квадратных скобок.
        """
        if isinstance(key, slice):
            return list(self.dictionary.keys())[key.start: key.stop: key.step]
        else:
            return list(self.dictionary.keys())[key]
    
    def __bool__(self):
        """ Класс ведет себя как булевская переменная. Проверяет было ли что-нибудь закешировано.
        """
        return len(self.dictionary)!=0
    
    def __call__(self):
        """ Объект класса можно "вызвать" как функцию. Можно просто переопределить оператор "круглые скобки".
        """
        print("-=* Overall results for FasterMorphology2*=-\nCash size: "+str(len(self.cash))+"\nDictionary size: "+str(len(self.dictionary)))
        return self

In [50]:
@functools.singledispatch
def cosineSimilarity(a, b): # Говорим, что у нас есть функция, которую мы будем расширять.
    return 0
    
@cosineSimilarity.register(list) # Навешиваем на нее декоратор, который будет вызываться только если первый параметр имеет тип list.
def _(a, b):
    if len(a)==0 or len(b)==0 or len(a)!=len(b): # Длины векторов в этом случае должны совпадать.
        return 0
    sumab=sum([na*nb for na, nb in zip(a, b)])
    suma2=sum([na*na for na in a])
    sumb2=sum([nb*nb for nb in b])
    return sumab/math.sqrt(suma2*sumb2)
    
@cosineSimilarity.register(dict) # Еще один декоратор для типа dict.
def _(a, b):
    if len(a.keys())==0 or len(b.keys())==0: # Вектора должны хранить хоть что-то.
        return 0
    sumab=sum([a[na]*b[na] for na in set(a.keys())&set(b.keys())])
    suma2=sum([a[na]*a[na] for na in a.keys()])
    sumb2=sum([b[nb]*b[nb] for nb in b.keys()])
    return sumab/math.sqrt(suma2*sumb2)  

@cosineSimilarity.register(np.ndarray) # И декоратор для типа np.array.
def _(a, b):
    if len(a)==0 or len(b)==0 or len(a)!=len(b): # Длины векторов в этом случае должны совпадать.
        return 0
    sumab=sum([na*nb for na, nb in zip(a, b)])
    suma2=sum([na*na for na in a])
    sumb2=sum([nb*nb for nb in b])
    return sumab/math.sqrt(suma2*sumb2)

@functools.singledispatch
def JaccardCoefficient(a, b): # Повторяем для коэффициента Жаккара.
    return 0

@JaccardCoefficient.register(list)
def _(a, b):
    if len(a)==0 or len(b)==0 or len(a)!=len(b): # Длины векторов в этом случае должны совпадать.
        return 0
    union=len(a)-[aa*bb for aa, bb in zip(a, b)].count(0)
    intersection=len(a)-[aa+bb for aa, bb in zip(a, b)].count(0)
    return union/intersection

@JaccardCoefficient.register(dict)
def _(a, b):
    if len(a.keys())==0 or len(b.keys())==0: # Вектора должны хранить хоть что-то.
        return 0
    return len(set(a.keys())&set(b.keys()))/len(set(a.keys())|set(b.keys()))

@JaccardCoefficient.register(np.ndarray)
def _(a, b):
    if len(a)==0 or len(b)==0 or len(a)!=len(b): # Длины векторов в этом случае должны совпадать.
        return 0
    union=len(a)-[aa*bb for aa, bb in zip(a, b)].count(0)
    intersection=len(a)-[aa+bb for aa, bb in zip(a, b)].count(0)
    return union/intersection

In [51]:
# Посмотрим как ведет себя косинус для разных типов.
a=[1,2,3]
b=[3,2,1]

print(cosineSimilarity(a, b))

a={1:1,2:2,3:3}
b={1:3,2:2,3:1}

print(cosineSimilarity(a, b))
   
a=np.array(([1,2,3]))
b=np.array(([3,2,1]))

print(cosineSimilarity(a, b))

0.7142857142857143
0.7142857142857143
0.7142857142857143


А теперь посмотрим как будут вести себя векторы от нашей кешированной морфологии.

In [52]:
fasterWP=FasterMorphology2()
fasterMart=FasterMorphology2()
fasterEnd=FasterMorphology2()
vctWP=fasterWP<<textWP
fasterMart+=fasterWP
vctMart=fasterMart<<textMart
fasterEnd+=fasterWP
fasterEnd+=fasterMart
vctEnd=fasterEnd<<textEnd

In [53]:
print("Cosine similarity of War and Peace and Ender's Game", cosineSimilarity(vctWP, vctEnd))
print("Cosine similarity of Martian and Ender's Game", cosineSimilarity(vctMart, vctEnd))
print("Cosine similarity of War and Peace and Martian", cosineSimilarity(vctMart, vctWP))

print("Jaccard similarity of War and Peace and Ender's Game", JaccardCoefficient(vctWP, vctEnd))
print("Jaccard similarity of Martian and Ender's Game", JaccardCoefficient(vctMart, vctEnd))
print("Jaccard similarity of War and Peace and Martian", JaccardCoefficient(vctMart, vctWP))

Cosine similarity of War and Peace and Ender's Game 0.8877558800566507
Cosine similarity of Martian and Ender's Game 0.8558887385483673
Cosine similarity of War and Peace and Martian 0.8042099465818117
Jaccard similarity of War and Peace and Ender's Game 0.24524534043362495
Jaccard similarity of Martian and Ender's Game 0.3274018379281537
Jaccard similarity of War and Peace and Martian 0.2114193992203623


Да, при таком маленьком пересечении лексики слишком большое совпадение косинусных мер. Давайте посчитаем только для значимых частей речи.

In [54]:
# Все декораторы вынесены из определения класса, так как не дело это класса считать косинусную меру. Она сама по себе.
# А еще я заменил действие оператора << , теперь он возвращает словарь. Так все-таки проще считать косинусную меру.
# Заведен список значимых частей речи, в соответствии с которым проводится фильтрация результатов.
# Теперь у нас есть два закешированных списка: интересующей нас части речи и остальные.
# Как следствие, пришлось переписать все части, которые касаются кеширования результатов.
class FasterMorphology2:
    """ Класс для быстрого морфологического анализа текстов и их векторизации.
    """
    
    def __init__(self): # Функция инициализации объекта после его создания.
        self.morpho=pymorphy2.MorphAnalyzer()
        self.cashPos={} # Теперь надо различать слова, которые нам нравятся, не нравятся икоторые не встретились.
        self.cashNeg=[]
        self.dictionary={} # Добавим словарь для запоминания, на каком месте вектора находится какая начальная форма.
        self.imPoS=['ADJF', 'NOUN', 'VERB', 'INFN', 'PRTF', 'GRND']
        
    def analyzeWords(self, words):
        """ Проводит морфологический анализ списка токенов words.
            Возвращает список начальных форм слов.
        """
        res=[]
        for word in words:
            if word in self.cashPos: # Сперва ищем очередное слово в кеше.
                res.append(self.cashPos[word])
            if word in self.cashNeg: # Сперва ищем очередное слово в кеше.
                pass
            else: # Если его там нет, проводим морфологический анализ и кешируем.
                r=self.morpho.parse(word)
                if r[0].tag.POS in self.imPoS:
                    r=r[0].normal_form
                    res.append(r)
                    self.cashPos[word]=r
                    if r not in self.dictionary: # Также для каждой начальной формы запоминаем ее позицию в векторе.
                        self.dictionary[r]=len(self.dictionary)
                else:
                    self.cashNeg.append(word)
        return res
    
    def breakByWords(self, text):
        """ Разбивает текст на русские слова.
        """
        return [w[0].lower() for w in re.findall("([А-ЯЁа-яё]+(-[А-ЯЁа-яё]+)*)", text)]
    
    def analyzeText(self, text):
        """ Проводит морфологический анализ строки с текстом text. 
            Выделяет из нее слова, написанные русской кириллицей.
            Возвращает список начальных форм слов.
        """
        words=self.breakByWords(text)
        return self.analyzeWords(words)
        
    # Вообще-то тоже самое умеет Counter, но ему надо сперва привести слова к начальной форме.
    def vectorizeAsDict(self, words):
        """ Возвращает векторное разреженное представление текста в виде словаря.
            Текст передается как список токенов words.
            Вместо позиции для индексации используется само слово.
            Возвращает словарь с начальными формами в ключах и частотами этих форм.
        """
        # Если это был текст - разбиваем на слова.
        if isinstance(words, str):
            words=self.breakByWords(words)

        vct={}
        for word in words: # Для каждого слова прповодим анализ.
            if word in self.cashPos: # Это закешированное слово со значимой частью речи.
                vct[self.cashPos[word]]=vct.get(self.cashPos[word], 0)+1 # Считаем частоты слов.
            elif word in self.cashNeg: # Это закешированное слово не со значимой частью речи.
                pass
            else:
                r=self.morpho.parse(word)
                if r[0].tag.POS in self.imPoS:
                    r=r[0].normal_form
                    self.cashPos[word]=r
                    vct[r]=1
                    if r not in self.dictionary:
                        self.dictionary[r]=len(self.dictionary)
                else:
                    self.cashNeg.append(word) # Мы ничего не хотим знать про это слово.

        return vct
    
    def clearDict(self):
        """ Очищает словарь. Вдруг надо пересчитать так как изменилась размерность пространства.
        """
        self.dictionary={}
    
    def formDict(self, texts):
        """ Сформировать словарь по тексту не формируя разметку текста.
        """
        for text in texts:
            for word in text:
                if word not in self.cashPos and word not in self.cashNeg:
                    r=self.morpho.parse(word)
                    if r[0].tag.POS in self.imPoS:
                        r=r[0].normal_form
                        self.cashPos[word]=r
                        if r not in self.dictionary:
                            self.dictionary[r]=len(self.dictionary)
                    else:
                        self.cashNeg.append(word) # Мы ничего не хотим знать про это слово.
    
    def vectorizeAsList(self, words2):
        """ Возвращает векторное представление текста в виде плотного списка (включает нули).
            Текст передается как список токенов words.
            Позиция каждого слова в векторе определяется числом, хранимым в dictionary.
            Возвращает словарь с начальными формами в ключах и частотами этих форм.
        """
        # Если это был текст - разбиваем на слова.
        if isinstance(words2, str):
            words=self.breakByWords(words2)
        else:
            words=words2

        # Сперва обновляем dictionary.
        for word in words:
            if word in self.cashNeg:
                pass
            elif word not in self.cashPos:
                r=self.morpho.parse(word)
                if r[0].tag.POS in self.imPoS:
                    r=r[0].normal_form
                    self.cashPos[word]=r
                    if r not in self.dictionary:
                        self.dictionary[r]=len(self.dictionary.keys())
                else:
                    self.cashNeg.append(word) # Мы ничего не хотим знать про это слово.
        # Теперь, когда все слова есть в кеше и словаре и известен размер вектора, можно приступать к векторизации.
        vct=[0 for _ in self.dictionary]
        for word in words:
            if word in self.cashPos:
                vct[self.dictionary[self.cashPos[word]]]+=1
        return vct
    
    def vectorizeAsList2(self, words):
        """ Возвращает векторное представление текста в виде плотного списка (включает нули).
            Текст передается как список токенов words. В вектор включаются только слова, находящиес в словаре.
            Позиция каждого слова в векторе определяется числом, хранимым в dictionary.
            Возвращает словарь с начальными формами в ключах и частотами этих форм.
        """
        vct=[0 for _ in self.dictionary]
        for word in words:
            if word in self.cashPos:
                vct[self.dictionary[self.cashPos[word]]]+=1
        return vct

    def vectorizeAsArray(self, words):
        """ Возвращает векторное представление текста в виде плотного массива (включает нули).
            Текст передается как список токенов words.
            Позиция каждого слова в векторе определяется числом, хранимым в dictionary.
            Возвращает словарь с начальными формами в ключах и частотами этих форм.
        """
        # Если это был текст - разбиваем на слова.
        if isinstance(words, str):
            words=self.breakByWords(words)

        # Сперва обновляем dictionary.
        for word in words:
            if word in self.cashNeg:
                pass
            elif word not in self.cashPos:
                r=self.morpho.parse(word)
                if r[0].tag.POS in self.imPoS:
                    r=r[0].normal_form
                    self.cashPos[word]=r
                    if r not in self.dictionary:
                        self.dictionary[r]=len(self.dictionary.keys())
                else:
                    self.cashNeg.append(word) # Мы ничего не хотим знать про это слово.
        # Теперь, когда все слова есть в кеше и словаре и известен размер вектора, можно приступать к векторизации.
        vct=np.zeros((len(self.dictionary)))
        for word in words:
            if word in self.cashPos:
                vct[self.dictionary[self.cashPos[word]]]+=1
        return vct

    def __iadd__(self, other):
        """ Оператор добавления словаря от другого объекта.
        """
        self.cashPos.update(other.cashPos)
        self.cashNeg=list(set(self.cashNeg)|set(other.cashNeg))
        for word in set(other.dictionary.keys())-set(self.dictionary.keys()):
            self.dictionary[word]=len(self.dictionary)
        return self
    
    def __lshift__(self, text):
        """ Оператор возвращает векторное представление текста в виде словаря.
        """
        return self.vectorizeAsDict(text)
        
    def __repr__(self):
        """ Текстовое представление объекта.
        """
        return "Object of class FasterMorphology2 <"+str(id(self))+ \
                ">\nPositive cash size: "+str(len(self.cashPos))+ \
                ">\nNegative cash size: "+str(len(self.cashNeg))+ \
                "\nDictionary size: "+str(len(self.dictionary))
    
    def __getitem__(self, key):
        """ Возвращает элемент словаря при помощи квадратных скобок.
        """
        if isinstance(key, slice):
            return list(self.dictionary.keys())[key.start: key.stop: key.step]
        else:
            return list(self.dictionary.keys())[key]
    
    def __bool__(self):
        """ Класс ведет себя как булевская переменная. Проверяет было ли что-нибудь закешировано.
        """
        return len(self.dictionary)!=0
    
    def __call__(self):
        """ Объект класса можно "вызвать" как функцию. Можно просто переопределить оператор "круглые скобки".
        """
        print("-=* Overall results for FasterMorphology2*=-\nPositive cash size: "+str(len(self.cashPos))+ \
              "Negative cash size: "+str(len(self.cashNeg))+ \
              "\nDictionary size: "+str(len(self.dictionary)))
        return self


In [55]:
fasterWP=FasterMorphology2()
fasterMart=FasterMorphology2()
fasterEnd=FasterMorphology2()
vctWP=fasterWP<<textWP
fasterMart+=fasterWP
vctMart=fasterMart<<textMart
fasterEnd+=fasterWP
fasterEnd+=fasterMart
vctEnd=fasterEnd<<textEnd

In [56]:
"""
Old version
Cosine similarity of War and Peace and Ender's Game 0.8877558800566507
Cosine similarity of Martian and Ender's Game 0.8558887385483673
Cosine similarity of War and Peace and Martian 0.8042099465818117
Jaccard similarity of War and Peace and Ender's Game 0.24524534043362495
Jaccard similarity of Martian and Ender's Game 0.3274018379281537
Jaccard similarity of War and Peace and Martian 0.2114193992203623
"""

print("Cosine similarity of War and Peace and Ender's Game", cosineSimilarity(vctWP, vctEnd))
print("Cosine similarity of Martian and Ender's Game", cosineSimilarity(vctMart, vctEnd))
print("Cosine similarity of War and Peace and Martian", cosineSimilarity(vctMart, vctWP))

print("Jaccard similarity of War and Peace and Ender's Game", JaccardCoefficient(vctWP, vctEnd))
print("Jaccard similarity of Martian and Ender's Game", JaccardCoefficient(vctMart, vctEnd))
print("Jaccard similarity of War and Peace and Martian", JaccardCoefficient(vctMart, vctWP))

Cosine similarity of War and Peace and Ender's Game 0.6191383100993432
Cosine similarity of Martian and Ender's Game 0.6791956209782578
Cosine similarity of War and Peace and Martian 0.6441131111259766
Jaccard similarity of War and Peace and Ender's Game 0.23424136838770984
Jaccard similarity of Martian and Ender's Game 0.30856387498819754
Jaccard similarity of War and Peace and Martian 0.19728585276261415


При небольшом уменьшении меры Жаккара косинусная мера значительно уменьшилась. Помимо этого, поменялась и схожесть текстов: теперь "Марсианин" больше похож на "Игру Эндера", чем на "Войну и мир".